In [45]:
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import numpy as np
df_train_labels_original = pd.read_csv('train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id').apply(lambda x: x-1)
df_train_values_original = pd.read_csv('train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'int64',
    'geo_level_3_id':'int64', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id').drop(columns=['geo_level_3_id'])


pd.options.display.float_format = '{:20,.2f}'.format

In [46]:
df = df_train_values_original.join(df_train_labels_original,how="inner")
df

,geo_level_1_id,geo_level_2_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,2,30,6,5,t,r,n,f,...,0,0,0,0,0,0,0,0,0,2
28830,8,900,2,10,8,7,o,r,n,x,...,0,0,0,0,0,0,0,0,0,1
94947,21,363,2,10,5,5,t,r,n,f,...,0,0,0,0,0,0,0,0,0,2
590882,22,418,2,10,6,5,t,r,n,f,...,0,0,0,0,0,0,0,0,0,1
201944,11,131,3,30,8,9,t,r,n,f,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1,55,6,3,n,r,n,f,...,0,0,0,0,0,0,0,0,0,1
669485,17,715,2,0,6,5,t,r,n,f,...,0,0,0,0,0,0,0,0,0,2
602512,17,51,3,55,6,7,t,r,q,f,...,0,0,0,0,0,0,0,0,0,2


In [47]:
def mean_encode(dataframe, column_name):
    new_column_names = {
        0: column_name+'_0',
        1: column_name+'_1',
        2: column_name+'_2',
    }
    cross = pd.crosstab(dataframe[column_name], dataframe['damage_grade']).rename(columns=new_column_names)
    prob = cross.divide(cross.apply('sum',axis=1),axis=0).reset_index()
    return dataframe.reset_index().merge(prob,on=column_name).set_index('building_id').drop(columns=[column_name])

In [48]:
def one_hot_encode_data(dataframe, column_name):
    dummies = pd.get_dummies(dataframe[column_name])
    rename_columns = {}
    for column in dummies.columns.values:
        rename_columns[column] = column_name + '_' + column
    return dataframe.drop(columns=[column_name]).join(dummies.rename(columns=rename_columns))

In [49]:
#df = mean_encode(df, 'geo_level_2_id')
df = one_hot_encode_data(df,'land_surface_condition')
df = one_hot_encode_data(df,'foundation_type')
df = one_hot_encode_data(df,'roof_type')
df = one_hot_encode_data(df,'ground_floor_type')
df = one_hot_encode_data(df,'other_floor_type')
df = one_hot_encode_data(df,'position')
df = one_hot_encode_data(df,'plan_configuration')
df = one_hot_encode_data(df,'legal_ownership_status')
df = one_hot_encode_data(df,'geo_level_1_id')
df

,geo_level_2_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,...,geo_level_1_id_28,geo_level_1_id_29,geo_level_1_id_3,geo_level_1_id_30,geo_level_1_id_4,geo_level_1_id_5,geo_level_1_id_6,geo_level_1_id_7,geo_level_1_id_8,geo_level_1_id_9
building_id,,,,,,,,,,,,,,,,,,,,,
802906,487,2,30,6,5,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
28830,900,2,10,8,7,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
94947,363,2,10,5,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
590882,418,2,10,6,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201944,131,3,30,8,9,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,1335,1,55,6,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
669485,715,2,0,6,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
602512,51,3,55,6,7,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
train_df, target = (df.drop(columns=['damage_grade'])[:int(len(df)/2)], df['damage_grade'][:int(len(df)/2)])
test_df, test_target = (df.drop(columns=['damage_grade'])[int(len(df)/2):], df['damage_grade'][int(len(df)/2):])
train_df

,geo_level_2_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,...,geo_level_1_id_28,geo_level_1_id_29,geo_level_1_id_3,geo_level_1_id_30,geo_level_1_id_4,geo_level_1_id_5,geo_level_1_id_6,geo_level_1_id_7,geo_level_1_id_8,geo_level_1_id_9
building_id,,,,,,,,,,,,,,,,,,,,,
802906,487,2,30,6,5,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
28830,900,2,10,8,7,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
94947,363,2,10,5,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
590882,418,2,10,6,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201944,131,3,30,8,9,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168990,1132,2,5,7,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
966340,36,1,5,7,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
842927,755,2,35,6,7,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [51]:
test_df

,geo_level_2_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,...,geo_level_1_id_28,geo_level_1_id_29,geo_level_1_id_3,geo_level_1_id_30,geo_level_1_id_4,geo_level_1_id_5,geo_level_1_id_6,geo_level_1_id_7,geo_level_1_id_8,geo_level_1_id_9
building_id,,,,,,,,,,,,,,,,,,,,,
666622,1023,1,10,10,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
194057,463,2,5,8,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
736347,1131,1,5,16,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69992,1030,1,5,4,4,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
133103,1115,2,15,10,5,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,1335,1,55,6,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
669485,715,2,0,6,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
602512,51,3,55,6,7,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=4, shuffle=True).split(train_df, target)
generic_prob = pd.crosstab(train_df['geo_level_2_id'], target).applymap(lambda x: 1/3)
train_df_after = pd.DataFrame()

for training_index, validation_index in folds:
    cross = pd.crosstab(train_df.iloc[training_index]['geo_level_2_id'], target.iloc[training_index])
    prob = cross.divide(cross.apply('sum',axis=1),axis=0).reset_index()
    result = train_df.iloc[validation_index].reset_index().merge(prob,on='geo_level_2_id',how='left')
    result = result.fillna(1/3).set_index('building_id')
    train_df_after = train_df_after.append(result)
    
global_mean = train_df_after.groupby('geo_level_2_id').mean()[[0,1,2]].reset_index()
test_df_after = test_df.reset_index().merge(global_mean,on='geo_level_2_id',how='left')
test_df_after = test_df_after.fillna(1/3).drop(columns=['geo_level_2_id'])
test_df_after

,building_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,...,geo_level_1_id_30,geo_level_1_id_4,geo_level_1_id_5,geo_level_1_id_6,geo_level_1_id_7,geo_level_1_id_8,geo_level_1_id_9,0,1,2
0,666622,1,10,10,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0.41,0.51,0.07
1,194057,2,5,8,6,0,0,0,0,0,...,0,0,0,0,0,1,0,0.20,0.42,0.38
2,736347,1,5,16,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0.29,0.66,0.05
3,69992,1,5,4,4,0,0,1,0,0,...,0,0,0,0,0,0,0,0.13,0.35,0.52
4,133103,2,15,10,5,0,1,0,0,0,...,0,0,0,0,1,0,0,0.03,0.83,0.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130296,688636,1,55,6,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0.17,0.76,0.07
130297,669485,2,0,6,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0.01,0.05,0.94
130298,602512,3,55,6,7,0,1,0,0,0,...,0,0,0,0,0,0,0,0.02,0.05,0.93
130299,151409,2,10,14,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0.50,0.47,0.03


In [54]:
cardinal_train = train_df.copy()
cardinal_test = test_df.copy()
train_df = train_df.drop(columns=['geo_level_2_id'])
test_df = test_df.drop(columns=['geo_level_2_id'])

In [56]:
dataset = tf.data.Dataset.from_tensor_slices((train_df.values, target.values))
dataset_after = tf.data.Dataset.from_tensor_slices((train_df_after.values, target.values))
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 2 30  6  5  1  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  1  0  0  1  0  0  1  0  0  1  0  0  0  0  0  1  0  0  0
  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0], Target: 2
Features: [ 2 10  8  7  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  1  0  0  0  1  0  0  1  0  0  0  0  0  1  0  0  1  0  0  0
  0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0], Target: 1
Features: [ 2 10  5  5  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  1  0  0  1  0  0  1  0  0  1  0  0  0  0  0  0  0  1  0
  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0], Target: 2
Features: [ 2 10  6  5  0  1  0  0  0  0  1  1  0  0  0  1

In [57]:
def compile_model(hp):
    layers = []
    l_amount = hp.Int('l_amount', min_value=3, max_value=9, step=1)
    l_size = hp.Int('l_size', min_value=200, max_value=1000, step=100)
    for x in range(l_amount):
        layers.append(tf.keras.layers.Dense(l_size, activation='relu'))
    layers.append(tf.keras.layers.Dense(units=3, activation='softmax'))
    model = tf.keras.Sequential(layers)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    opt = tf.keras.optimizers.Adam(clipnorm=1.0)
    model.compile(optimizer=opt,
                  loss=loss_fn,
                  metrics=['accuracy'])
    return model

In [58]:
tuner = kt.BayesianOptimization(compile_model,
                                objective='val_accuracy',
                                max_trials=25,
                               )


In [59]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

tuner.search(
    train_df.to_numpy(), 
    target.to_numpy(), 
    epochs=25, 
    validation_data=(test_df, test_target),
    batch_size=128,
    callbacks=[stop_early],
)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


Trial 25 Complete [00h 02m 51s]
val_accuracy: 0.6888742446899414

Best val_accuracy So Far: 0.6899102926254272
Total elapsed time: 01h 36m 49s
INFO:tensorflow:Oracle triggered exit


In [60]:
best_hps.get('l_size')

500

In [61]:
best_hps.get('l_amount')

9

In [62]:
save_best = tf.keras.callbacks.ModelCheckpoint('modelos/NNModelBY_Best', save_best_only=True)

best_model = compile_model(best_hps)
best_model.fit(dataset.batch(128), epochs=200, validation_data=(test_df, test_target), callbacks=[save_best])

Epoch 1/200
1018/1018 [==============================] - 21s 20ms/step - loss: 0.7389 - accuracy: 0.6425 - val_loss: 0.7156 - val_accuracy: 0.6555
INFO:tensorflow:Assets written to: modelos\NNModelBY_Best\assets
Epoch 2/200
1018/1018 [==============================] - 20s 19ms/step - loss: 0.7019 - accuracy: 0.6677 - val_loss: 0.6931 - val_accuracy: 0.6767
INFO:tensorflow:Assets written to: modelos\NNModelBY_Best\assets
Epoch 3/200
1018/1018 [==============================] - 22s 22ms/step - loss: 0.6934 - accuracy: 0.6739 - val_loss: 0.6904 - val_accuracy: 0.6719
INFO:tensorflow:Assets written to: modelos\NNModelBY_Best\assets
Epoch 4/200
1018/1018 [==============================] - 24s 23ms/step - loss: 0.6898 - accuracy: 0.6770 - val_loss: 0.6819 - val_accuracy: 0.6827
INFO:tensorflow:Assets written to: modelos\NNModelBY_Best\assets
Epoch 5/200
1018/1018 [==============================] - 21s 21ms/step - loss: 0.6844 - accuracy: 0.6800 - val_loss: 0.6807 - val_accuracy: 0.6839
INFO:

1018/1018 [==============================] - 19s 18ms/step - loss: 0.4240 - accuracy: 0.8048 - val_loss: 1.2978 - val_accuracy: 0.6427
Epoch 53/200
1018/1018 [==============================] - 19s 18ms/step - loss: 0.4167 - accuracy: 0.8088 - val_loss: 1.2569 - val_accuracy: 0.6352
Epoch 54/200
1018/1018 [==============================] - 19s 18ms/step - loss: 0.4135 - accuracy: 0.8115 - val_loss: 1.3258 - val_accuracy: 0.6433
Epoch 55/200
1018/1018 [==============================] - 19s 18ms/step - loss: 0.4116 - accuracy: 0.8123 - val_loss: 1.3291 - val_accuracy: 0.6435
Epoch 56/200
1018/1018 [==============================] - 19s 19ms/step - loss: 0.4037 - accuracy: 0.8151 - val_loss: 1.2492 - val_accuracy: 0.6493
Epoch 57/200
1018/1018 [==============================] - 21s 21ms/step - loss: 0.4001 - accuracy: 0.8161 - val_loss: 1.3337 - val_accuracy: 0.6363
Epoch 58/200
1018/1018 [==============================] - 20s 20ms/step - loss: 0.3996 - accuracy: 0.8175 - val_loss: 1.3986 

1018/1018 [==============================] - 20s 20ms/step - loss: 0.2526 - accuracy: 0.8896 - val_loss: 1.5240 - val_accuracy: 0.6292
Epoch 108/200
1018/1018 [==============================] - 20s 20ms/step - loss: 0.2525 - accuracy: 0.8915 - val_loss: 1.5111 - val_accuracy: 0.6298
Epoch 109/200
1018/1018 [==============================] - 20s 20ms/step - loss: 0.2479 - accuracy: 0.8928 - val_loss: 1.7001 - val_accuracy: 0.6297
Epoch 110/200
1018/1018 [==============================] - 21s 20ms/step - loss: 0.2437 - accuracy: 0.8940 - val_loss: 1.6958 - val_accuracy: 0.6310
Epoch 111/200
1018/1018 [==============================] - 21s 21ms/step - loss: 0.2455 - accuracy: 0.8940 - val_loss: 1.5570 - val_accuracy: 0.6234
Epoch 112/200
1018/1018 [==============================] - 20s 20ms/step - loss: 0.2520 - accuracy: 0.8970 - val_loss: 2.0041 - val_accuracy: 0.6264
Epoch 113/200
1018/1018 [==============================] - 20s 20ms/step - loss: 0.2425 - accuracy: 0.8959 - val_loss: 1

1018/1018 [==============================] - 19s 19ms/step - loss: 0.1736 - accuracy: 0.9301 - val_loss: 1.7298 - val_accuracy: 0.6236
Epoch 163/200
1018/1018 [==============================] - 19s 19ms/step - loss: 0.1723 - accuracy: 0.9301 - val_loss: 1.9858 - val_accuracy: 0.6181
Epoch 164/200
1018/1018 [==============================] - 20s 20ms/step - loss: 0.1679 - accuracy: 0.9326 - val_loss: 1.8679 - val_accuracy: 0.6257
Epoch 165/200
1018/1018 [==============================] - 21s 20ms/step - loss: 0.1758 - accuracy: 0.9296 - val_loss: 1.8452 - val_accuracy: 0.6241
Epoch 166/200
1018/1018 [==============================] - 19s 19ms/step - loss: 0.1691 - accuracy: 0.9329 - val_loss: 2.0165 - val_accuracy: 0.6189
Epoch 167/200
1018/1018 [==============================] - 20s 19ms/step - loss: 0.1774 - accuracy: 0.9308 - val_loss: 1.7487 - val_accuracy: 0.6209
Epoch 168/200
1018/1018 [==============================] - 20s 20ms/step - loss: 0.1715 - accuracy: 0.9324 - val_loss: 1

In [ ]:
best_model.save('modelos/NNModelBY')

In [64]:
tf.keras.models.load_model('modelos/NNModelBY_Best').evaluate(test_df, test_target)

4072/4072 [==============================] - 12s 3ms/step - loss: 0.6748 - accuracy: 0.6856


[0.6748371720314026, 0.6856278777122498]